# ***Импорты***

In [1]:
!pip install --upgrade gensim
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaMulticore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# !pip install --upgrade numpy
!pip install pyLDAvis
# !pip install transformers
!pip install transformers==4.28.0 accelerate
!pip install datasets
# !pip install --upgrade gensim
import pandas as pd

import nltk 
# nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
# nltk.download('movie_reviews')
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# from gensim.models.ldamodel import LdaModel
# from gensim.models.coherencemodel import CoherenceModel
from sklearn.model_selection import train_test_split 
from datasets import Dataset
import pyLDAvis.gensim_models 
import pyLDAvis
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig, pipeline


from tqdm.notebook import tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 110.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.3 MB/s eta 0:00:0

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils

# ***Часть №1***

## ***Предобработка датасета***

In [3]:
covid = pd.read_csv("/content/covid19_tweets.csv")
covid

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24620,Abogado.com.ph,NaN,NaN,2017-01-20 07:23:27,10711,38,85,False,2020-07-26 02:12:27,Former COMELEC Chair tests positive for COVID-...,"['AbogadoSnitch', 'covid19']",TweetDeck,False
24621,Derik Van Derbeken,"Los Angeles, CA","Photog, VO, Foodie, Amateur Sleuth,💖-Rusty G d...",2009-03-12 23:01:59,409,2997,18143,False,2020-07-26 02:12:25,@chipfranklin He wouldn't actually take office...,NaN,Twitter for Android,False
24622,__________,"Cincinnati, OH","Big heart, sharp tongue. ...",2009-03-01 04:17:32,150,642,9992,False,2020-07-26 02:12:20,@AutographsToYou @Dee_Wallace This is such a b...,NaN,Twitter for Android,False
24623,Shannon,"Ohio, USA",Capricorn. 24.,2016-12-04 23:54:12,15,44,834,False,2020-07-26 02:12:20,On the left is the amount of suicide deaths fo...,NaN,Twitter for Android,False


In [5]:
covid = covid[["text"]]  # Оставляем только поле с текстом
covid

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text
0,If I smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,@brookbanktv The one gift #COVID19 has give me...
4,25 July : Media Bulletin on Novel #CoronaVirus...
...,...
24620,Former COMELEC Chair tests positive for COVID-...
24621,@chipfranklin He wouldn't actually take office...
24622,@AutographsToYou @Dee_Wallace This is such a b...
24623,On the left is the amount of suicide deaths fo...


In [6]:
def remove_emoji(string):                                                       # Параметры для запуска функции: строка (в нашем случае это текст записи)                                                
  """Функция для очистки строки от эмодзи"""
  emoji_pattern = re.compile("[" u"\U0001F600-\U0001F64F"                       # Сбор всех возможных смайликов и эмодзи  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f" 
                               u"\u3030"
                               u"\u23F0" "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)                                         # Возврат строки без смайликов

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
cleaned_texts = []                                                 # Создаём пустой список, в который поместим очищенные тексты
lemmatizer = WordNetLemmatizer()                                   # Создание экземпляра лемматизатора WordNetLemmatizer
stop_word = stopwords.words('english')                             # Загрузка стоп-слов для английского языка
prefix = ('@', '#', 'http', 'https', '//', "n't", "'s")            # Префиксы для удаления
for i in range(covid.shape[0]):                                    # Создаём цикл, который проходит по каждому тексту из датасета
  s = ''                                                           # Начальная строка пуста
  a = 0                                                            # Текущее слово из текста (изначально равно нулю)
  text = str(covid.text[i])                                             # Передаём текст в переменную text
  text = text.replace('…', '')                                     # Удаляем многоточие
  text = text.replace('\n', ' ')                                   # Удаляем символы '\n'
  text = text.replace('&amp', ' ')                                 # Удаляем символы '&amp'
  text = text.lower()                                              # Переводим текст в нижний регистр
  text = remove_emoji(text)                                        # Убираем из текста смайлики и эмоджи с помощью написанной выше функции
  text = word_tokenize(text)                                       # Проводим токенизацию
  text = [word for word in text if not word in stop_word]          # Удаляем стоп-слова
  text = [lemmatizer.lemmatize(word) for word in text]             # Проводим лемматизацию
  while a < len(text) - 1:                                         # Создаём цикл while, который проходит по каждому слову в тексте
    if not text[a].startswith(prefix):                             # Если слово начинается не с тех символов, которые лежат в переменной prefix, то
      if a != len(text) - 1:                                       # если рассматриваемое слово не последнее, то
        s += text[a] + ', '                                        # добавляем его в строку и ставим после него запятую
      else:                                                        # Если же рассматриваемое слово последнее, то
        s += text[a]                                               # просто добавляем его в строку
      a += 1                                                       # Переходим к следующему слову                                                                           
    else:                                                          # Если слово начинается с символов, которые лежат в переменной prefix, то
      if text[a].startswith('#') or text[a].startswith('@'):       # если слово начинается с символа '@' или '#', то
        a += 2                                                     # пропускаем это и следующее слово
      else:                                                        # Если же слово начинается не с символов '@' или '#', то
        a += 1                                                     # пропускаем это слово
  s = re.sub(r'[^\w\s]', '', s)                                    # Удаление знаков препинания
  s = ' '.join(s.split())                                          # Разделяем строку на слова и добавляем через пробел
  cleaned_texts.append(s)                                          # Добавляем строку в список

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
covid["full_clear"] = cleaned_texts  # Создаём новый столбец с очищенными текстами
covid                                # Вывод изменённого датасета

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-8-0ad6979e2e0c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid["full_clear"] = cleaned_texts  # Создаём новый столбец с очищенными текстами


,text,full_clear
0,If I smelled the scent of hand sanitizers toda...,smelled scent hand sanitizers today someone pa...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey would made sense player pay respect
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,trump never claimed hoax claim effort
3,@brookbanktv The one gift #COVID19 has give me...,one gift give appreciation simple thing always...
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july medium bulletin novel
...,...,...
24620,Former COMELEC Chair tests positive for COVID-...,former comelec chair test positive covid19
24621,@chipfranklin He wouldn't actually take office...,would actually take office till january 21st 1...
24622,@AutographsToYou @Dee_Wallace This is such a b...,better idea ms wallace endangering loved one
24623,On the left is the amount of suicide deaths fo...,left amount suicide death year right amount co...


## ***Словарь и частоты слов***

https://www.geeksforgeeks.org/nlp-gensim-tutorial-complete-guide-for-beginners/

In [9]:
from gensim import corpora
tokens = []                                       # Массив для токенов
for i in range(len(covid)):                       # Создаём цикл, который каждый текст
  token = word_tokenize(covid.full_clear[i])      # токенизирует и
  tokens.append(token)                            # добавляет все токены в массив
covid["tokens"] = tokens                          # Создаём новый столбец с токенами
my_dictionary = corpora.Dictionary(covid.tokens)  # Создаём словарь

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-9-d653d9f0c520>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid["tokens"] = tokens                          # Создаём новый столбец с токенами


In [10]:
corpus = [my_dictionary.doc2bow(doc, allow_update = True) for doc in covid.tokens]  # Создаём корпус на основе словаря

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
word_freq = [[(my_dictionary[id], frequence) for id, frequence in couple] for couple in corpus]  # Поиск встречаемости всех слов в предложении
print(word_freq[:2])                                                                                        # Вывод встречаемости

[[('hand', 1), ('intoxicated', 1), ('past', 1), ('sanitizers', 1), ('scent', 1), ('smelled', 1), ('someone', 1), ('think', 1), ('today', 1), ('would', 1)], [('would', 1), ('hey', 1), ('made', 1), ('pay', 1), ('player', 1), ('respect', 1), ('sense', 1)]]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## ***Модель LDA***

In [12]:
coherence = 0                                                                                                                              # Начальный коэффиент согласованности равен 0
for i in tqdm(range(1, 11)):                                                                                                               # Создаём цикл, который рассмотрит количество тем от 1 до 10
  model=LdaMulticore(corpus=corpus, id2word=my_dictionary, num_topics=i, random_state=200)                                                 # Обучаем модель LDA
  coherence_model = CoherenceModel(model=model, texts=covid["tokens"].tolist(), dictionary=my_dictionary, corpus=corpus, coherence='c_v')  # Обучаем модель согласованности
  new_coherence = coherence_model.get_coherence()                                                                                          # Находим для модели коэффициент согласованности 
  if new_coherence > coherence:                                                                                                            # Если он больше текущего, то
    coherence = new_coherence                                                                                                              # задаём переменной coherence новое значение и
    number = i
print()                                                                                                                             # выводим количество тем
print(coherence)
print(number)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 10/10 [03:40<00:00, 22.06s/it]


0.3393510583603389
3


**Лучший результат показала модель с *3* скрытыми темами**

In [13]:
model=LdaMulticore(corpus=corpus, id2word=my_dictionary, num_topics=number, random_state=100)                                            # Обучаем модель LDA на полученных данных
print('\nPerplexity:', model.log_perplexity(corpus))                                                                                     # Вывод перплексии
coherence_model = CoherenceModel(model=model, texts=covid["tokens"].tolist(), dictionary=my_dictionary, corpus=corpus, coherence='c_v')  # Обучаем модель согласованности на лучшей модели LDA
coherence_lda = coherence_model.get_coherence()                                                                                          # Находим коэффициент согласованности
print('\nCoherence Score:', coherence_lda)                                                                                               # Выводим коэффициент согласованности

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity: -8.707250339322494

Coherence Score: 0.33459632477955986


In [14]:
model.print_topics()  # Вывод тем

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.046*"case" + 0.023*"new" + 0.022*"death" + 0.013*"total" + 0.012*"positive" + 0.010*"day" + 0.008*"number" + 0.008*"last" + 0.007*"people" + 0.007*"india"'),
 (1,
  '0.008*"covid19" + 0.006*"get" + 0.006*"u" + 0.004*"new" + 0.004*"one" + 0.004*"2020" + 0.004*"time" + 0.004*"people" + 0.004*"july" + 0.003*"hospital"'),
 (2,
  '0.013*"mask" + 0.007*"people" + 0.006*"pandemic" + 0.004*"like" + 0.004*"u" + 0.004*"covid19" + 0.004*"american" + 0.004*"please" + 0.004*"home" + 0.004*"need"')]

In [15]:
pyLDAvis.enable_notebook()                                          # Включение библиотеки pyLDAvis
vis = pyLDAvis.gensim_models.prepare(model, corpus, my_dictionary)  # Подготовка к построению визуализации
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.095491  0.089461       1        1  39.043111
0      0.161583  0.011552       2        1  31.169154
1     -0.066092 -0.101013       3        1  29.787734, topic_info=            Term         Freq        Total Category  logprob  loglift
72          case  3193.000000  3193.000000  Default  30.0000  30.0000
39         death  1532.000000  1532.000000  Default  29.0000  29.0000
269          new  1883.000000  1883.000000  Default  28.0000  28.0000
232        total   879.000000   879.000000  Default  27.0000  27.0000
205     positive   833.000000   833.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
150  coronavirus   171.782489   842.042439   Topic3  -5.9060  -0.3785
302         like   160.819351   588.515111   Topic3  -5.9720  -0.0862
21         trump   154.032653   420.253756   Topic3  -6.0151   0.2074
512     pandemic   154.225719   752.195898   Topic3  -6.0139  -0.3735
226          day   151.372762   950.954943   Topic3  -6.0325  -0.6266

[275 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
330        1  0.101952      10
330        2  0.787808      10
330        3  0.111220      10
233        1  0.058135      13
233        2  0.920474      13
...      ...       ...     ...
3976       1  0.947408     yep
12170      2  0.972774  yester
3212       1  0.121330     yet
3212       2  0.109775     yet
3212       3  0.762649     yet

[545 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

# ***Часть №2***

## ***Предобработка датасета***

In [4]:
covid = pd.read_csv("/content/covid19_tweets.csv")
covid

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,AJIMATI AbdulRahman O.,"Ilorin, Nigeria",Animal Scientist|| Muslim|| Real Madrid/Chelsea,2013-12-30 18:59:19,412,1609,1062,False,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,['WearAMask'],Twitter for Android,False
179104,Jason,Ontario,When your cat has more baking soda than Ninja ...,2011-12-21 04:41:30,150,182,7295,False,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,['COVID19'],Twitter for Android,False
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,NaN,Twitter Web App,False
179106,Gary DelPonte,New York City,"Global UX UI Visual Designer. StoryTeller, Mus...",2009-10-27 17:43:13,1338,1111,0,False,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...",['COVID19'],Twitter for iPhone,False


In [5]:
covid = covid[["text"]]
covid

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text
0,If I smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,@brookbanktv The one gift #COVID19 has give me...
4,25 July : Media Bulletin on Novel #CoronaVirus...
...,...
179103,Thanks @IamOhmai for nominating me for the @WH...
179104,2020! The year of insanity! Lol! #COVID19 http...
179105,@CTVNews A powerful painting by Juan Lucena. I...
179106,"More than 1,200 students test positive for #CO..."


In [6]:
def remove_emoji(string):                                                       # Параметры для запуска функции: строка (в нашем случае это текст записи)                                                
  """Функция для очистки строки от эмодзи"""
  emoji_pattern = re.compile("[" u"\U0001F600-\U0001F64F"                       # Сбор всех возможных смайликов и эмодзи  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f" 
                               u"\u3030"
                               u"\u23F0" "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)                                         # Возврат строки без смайликов

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
cleaned_texts = []                                                 # Создаём пустой список, в который поместим очищенные тексты
lemmatizer = WordNetLemmatizer()                                   # Создание экземпляра лемматизатора WordNetLemmatizer
stop_word = stopwords.words('english')                             # Загрузка стоп-слов для английского языка
prefix = ('@', '#', 'http', 'https', '//', "n't", "'s")            # Префиксы для удаления
for i in range(covid.shape[0]):                                    # Создаём цикл, который проходит по каждому тексту из датасета
  s = ''                                                           # Начальная строка пуста
  a = 0                                                            # Текущее слово из текста (изначально равно нулю)
  text = covid.text[i]                                             # Передаём текст в переменную text
  text = text.replace('…', '')                                     # Удаляем многоточие
  text = text.replace('\n', ' ')                                   # Удаляем символы '\n'
  text = text.replace('&amp', ' ')                                 # Удаляем символы '&amp'
  text = text.lower()                                              # Переводим текст в нижний регистр
  text = remove_emoji(text)                                        # Убираем из текста смайлики и эмоджи с помощью написанной выше функции
  text = word_tokenize(text)                                       # Проводим токенизацию
  while a < len(text) - 1:                                         # Создаём цикл while, который проходит по каждому слову в тексте
    if not text[a].startswith(prefix):                             # Если слово начинается не с тех символов, которые лежат в переменной prefix, то
      if a != len(text) - 1:                                       # если рассматриваемое слово не последнее, то
        s += text[a] + ', '                                        # добавляем его в строку и ставим после него запятую
      else:                                                        # Если же рассматриваемое слово последнее, то
        s += text[a]                                               # просто добавляем его в строку
      a += 1                                                       # Переходим к следующему слову                                                                           
    else:                                                          # Если слово начинается с символов, которые лежат в переменной prefix, то
      if text[a].startswith('#') or text[a].startswith('@'):       # если слово начинается с символа '@' или '#', то
        a += 2                                                     # пропускаем это и следующее слово
      else:                                                        # Если же слово начинается не с символов '@' или '#', то
        a += 1                                                     # пропускаем это слово
  s = re.sub(r'[^\w\s]', '', s)                                    # Удаление знаков препинания
  s = ' '.join(s.split())                                          # Разделяем строку на слова и добавляем через пробел
  cleaned_texts.append(s)                                          # Добавляем строку в список

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
covid["full_clear"] = cleaned_texts
covid

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-8-ac7362c2ebe0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid["full_clear"] = cleaned_texts


,text,full_clear
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey and would it have made more sense to have ...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,trump never once claimed was a hoax we all cla...
3,@brookbanktv The one gift #COVID19 has give me...,the one gift has give me is an appreciation fo...
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july media bulletin on novel
...,...,...
179103,Thanks @IamOhmai for nominating me for the @WH...,thanks for nominating me for the challenge i n...
179104,2020! The year of insanity! Lol! #COVID19 http...,2020 the year of insanity lol
179105,@CTVNews A powerful painting by Juan Lucena. I...,a powerful painting by juan lucena it a tribut...
179106,"More than 1,200 students test positive for #CO...",more than 1200 students test positive for at m...


In [9]:
tokens = []
for i in range(len(covid)):
  token = word_tokenize(covid.full_clear[i])
  tokens.append(token)
covid["tokens"] = tokens

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-9-064294822c4e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid["tokens"] = tokens


In [10]:
final = covid[(covid['full_clear']) != '']             # Оставляем только те тексты, в которых имеется как минимум два слова (то есть строка содержит запятую)
final                                                  # Выводим датафрейм

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,full_clear,tokens
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...,"[if, i, smelled, the, scent, of, hand, sanitiz..."
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey and would it have made more sense to have ...,"[hey, and, would, it, have, made, more, sense,..."
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,trump never once claimed was a hoax we all cla...,"[trump, never, once, claimed, was, a, hoax, we..."
3,@brookbanktv The one gift #COVID19 has give me...,the one gift has give me is an appreciation fo...,"[the, one, gift, has, give, me, is, an, apprec..."
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july media bulletin on novel,"[25, july, media, bulletin, on, novel]"
...,...,...,...
179103,Thanks @IamOhmai for nominating me for the @WH...,thanks for nominating me for the challenge i n...,"[thanks, for, nominating, me, for, the, challe..."
179104,2020! The year of insanity! Lol! #COVID19 http...,2020 the year of insanity lol,"[2020, the, year, of, insanity, lol]"
179105,@CTVNews A powerful painting by Juan Lucena. I...,a powerful painting by juan lucena it a tribut...,"[a, powerful, painting, by, juan, lucena, it, ..."
179106,"More than 1,200 students test positive for #CO...",more than 1200 students test positive for at m...,"[more, than, 1200, students, test, positive, f..."


In [11]:
covid_sample = final.sample(n=10000)
covid_sample

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,full_clear,tokens
125190,"Yassin Juma, a Kenyan journalist who was arres...",yassin juma a kenyan journalist who was arrest...,"[yassin, juma, a, kenyan, journalist, who, was..."
155713,Was only trying to help...\n\n#UTI #LayCruiser...,was only trying to help,"[was, only, trying, to, help]"
31293,#VandeBharatMission | The repatriation flight ...,the repatriation flight schedule and other upd...,"[the, repatriation, flight, schedule, and, oth..."
173480,Very little is unchanged by the events of 2020...,very little is unchanged by the events of 2020...,"[very, little, is, unchanged, by, the, events,..."
73834,📝\nStructure of SARS-CoV-2 2'-O-methyltransfer...,structure of sarscov2 2omethyltransferase hete...,"[structure, of, sarscov2, 2omethyltransferase,..."
...,...,...,...
146579,"Spike of 57,982 cases and 941 deaths reported ...",spike of 57982 cases and 941 deaths reported i...,"[spike, of, 57982, cases, and, 941, deaths, re..."
85536,⁦@GOPconvention⁩ Gets Reality Check Over ‘Expe...,gets reality check over expectations exceeded ...,"[gets, reality, check, over, expectations, exc..."
119552,The Government and the media spread a great de...,the government and the media spread a great de...,"[the, government, and, the, media, spread, a, ..."
36065,@Juls4Change Guarantee: they're all plants. An...,guarantee they re all plants anyone can wear a...,"[guarantee, they, re, all, plants, anyone, can..."


## ***Дообучение базовой модели, файнтюнинг***

In [12]:
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
base_model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
print('Words in vocabulary:', base_tokenizer.vocab_size)

Words in vocabulary: 50257


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
def pretty_print(text, max_len_line=100):
    words = text.split(' ')
    len_line = 0
    line = ''
    for w in words:
        if w == '\n':
            print(line)
            line = ''
            continue
        if (len(line) + len(w)) > max_len_line:
            print(line)
            line = ''
        line += ' ' + w
    print(line)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
covid_sample["text"].tolist()[8]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'Trump announced a plan to gut #SocialSecurity instead of working with Congress to pass #COVID19 relief. \n\nWrite to… https://t.co/a7qfQKPHQi'

In [16]:
covid_sample["full_clear"].tolist()[8]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'trump announced a plan to gut instead of working with congress to pass relief write to'

In [17]:
from transformers import pipeline
#generator = pipeline('text-generation', model='gpt2')
#text_ids = base_tokenizer.encode(covid_sample["full_clear"].tolist(), max_length=10000, max_new_tokens=5000, truncation=True, return_tensors = 'pt')
text_ids = base_tokenizer.encode(covid_sample["full_clear"].tolist()[1], return_tensors = 'pt')
generated_text_samples = base_model.generate(text_ids, max_length=100)

for i, beam in enumerate(generated_text_samples):
    print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
    print()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: was only trying to help the people of the city.

"I'm not going to be a part of this," he said. "I'm not going to be a part of this. I'm not going to be a part of this. I'm not going to be a part of this. I'm not going to be a part of this. I'm not going to be a part of this. I'm not going to be a part of this. I'm not going to



In [18]:
covid_sample["full_clear"].tolist()[1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'was only trying to help'

In [19]:
generated_text_samples = base_model.generate(
    text_ids,
    max_length= 50,  
    num_beams=5,
    num_return_sequences= 5,
    early_stopping=True 
)

for i, beam in enumerate(generated_text_samples):
  pretty_print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
  print()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0: was only trying to help.

"I'm just trying to help," he said. "I'm just trying to help. I'm just
 trying to help. I'm just trying to help. I'm just trying to help. I'm

 1: was only trying to help.

"I'm just trying to help," he said. "I'm just trying to help. I'm just
 trying to help. I'm just trying to help. I'm just trying to help."



 2: was only trying to help.

"I'm just trying to help," she said. "I'm just trying to help. I'm just
 trying to help. I'm just trying to help. I'm just trying to help. I'm

 3: was only trying to help.

"I'm just trying to help," she said. "I'm just trying to help. I'm just
 trying to help. I'm just trying to help. I'm just trying to help."



 4: was only trying to help.

"I'm just trying to help," he said. "I'm just trying to help. I'm just
 trying to help. I'm just trying to help. I'm just trying to help. I just



In [20]:
generated_text_samples = base_model.generate(
    text_ids,
    max_length= 50,  
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences= 5,
    early_stopping=True 
)

for i, beam in enumerate(generated_text_samples):
  print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
  print()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: was only trying to help.

"I'm not going to give up," he said. "I don't want to lose my job. I want my kids to be able to go to college."

1: was only trying to help.

"I'm not going to give up," he said. "I don't want to lose my job."

2: was only trying to help.

"I'm not going to give up," he said. "I don't want to lose my job. I want my kids to have a better life."

3: was only trying to help.

"I'm not going to give up," he said. "I don't want to lose my job. I want my kids to be able to go to college and get a good job."

4: was only trying to help.

"I'm not going to give up," he said. "I don't want to lose my job. I want my kids to be able to go to college and get a good education."



In [21]:
generated_text_samples = base_model.generate(
    text_ids,
    max_length= 50,  
    do_sample=True,  
    top_k=0,
    num_return_sequences= 5
)

for i, beam in enumerate(generated_text_samples):
  pretty_print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
  print()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0: was only trying to help a friend if she wanted to. Although she received a bone marrow magnetised
 cell (MMC) from her donor, the patient not only had cancer showing up on the MRI, but also appeared
 to have a disordered release

 1: was only trying to help them reach some sort of reconciliation. However, they were also
 cognitively challenged in prose about how racism (high prescribed MSM treatment), which tends to
 create disparate opinions and experiences, may stimulate very negative attitudes toward minority
 people. However

 2: was only trying to help the band. He left on a tour bus today and he and his family are totally
 taken with the fun (he dumped the kids at YMCA numerous times, though).

"I like it. I passed out

 3: was only trying to help him, naked, as much of Ashley's pale skin as it was her face, and cares
 made things awkward between them.

At this point, Ashley heard Braden think of jogging down the
 street and did one

 4: was only trying to help serve these pe

In [22]:
generated_text_samples = base_model.generate(
    text_ids,
    max_length= 50,  
    do_sample=True,  
    top_k=0,
    temperature=0.9,
    num_return_sequences= 5
)

for i, beam in enumerate(generated_text_samples):
  pretty_print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
  print()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0: was only trying to help people if someone threatened him. So I wouldn't advise him to do it. I
 would recommend him to anyone who needs to know what happened to Don.

I would advise you to read
 many books on other of the

 1: was only trying to help her."

In 2000, Beck also provided emergency food to Gilbert that night,
 and her neighbor and second-floor neighbor read a newspaper about the hospital, asking for social
 services.

"Her initial response was,

 2: was only trying to help his friends. It's not like the police have any jurisdiction over the
 Peruvian cops. As a family, I'm scared to go outside and not see them.

DL: Do you fear for your
 safety or freedom

 3: was only trying to help with one rather large part, which looked the less beautiful than the
 other two because of the negro. There was no doubt about it, and, as I later explained why we kept
 seeing death incarnate at the same time,

 4: was only trying to help Andy in a live environment.

But, rightly or wrong

In [23]:
generated_text_samples = base_model.generate(
    text_ids,
    max_length= 50,  
    do_sample=True,  
    top_k=25,
    num_return_sequences= 5
)

for i, beam in enumerate(generated_text_samples):
  pretty_print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
  print()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0: was only trying to help a girl who was being abused," he said. He was shocked by the incident,
 particularly the fact that he is "in some way a parent."

"As a parent of child, it's just
 devastating, to

 1: was only trying to help people who had been victims on the streets, and they are also trying to
 get people out of jail so we can go to them and help them," she said. "The police have already done
 an amazing job on their own

 2: was only trying to help. He is trying to find his family, to make peace, to find peace for him
 now."

The father says he is so happy for his wife, her mother, and other children, for seeing the
 results of

 3: was only trying to help his son to be an entrepreneur and an entrepreneur in her mother's
 home.

In a letter to his mother, the mother writes: "We've been through a very difficult time and I
 know it will be difficult for

 4: was only trying to help those who had already been killed by the attackers in the other town. The
 militants also at

In [24]:
generated_text_samples = base_model.generate(
    text_ids,
    max_length= 50,  
    do_sample=True,  
    top_k=0,
    top_p=0.92,
    num_return_sequences= 5
)

for i, beam in enumerate(generated_text_samples):
  pretty_print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
  print()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0: was only trying to help my current wife, who went by the name Little Donald, pay her car loans.
 As her compensation by the auto companies was $600.00 an hour, she had to break even. We were at her
 house searching for

 1: was only trying to help her boyfriend, but I assume that did his part as he overcame his bit of
 sex because he is a human being.

Or maybe that is what Sadon likes to say. If you might consider
 him a "mere

 2: was only trying to help out with distribution. He did it for $30K because he knew if he didn't
 get funding from Amazon he'd die. He chose to live off people he felt were foolish enough to let his
 three kids grow up and

 3: was only trying to help out for friends at his university, and living at home.

His friend agreed
 to donate bitcoin to Uber to help cover the costs.

But after he caught wind of it, Glass stopped
 paying any amount to meet

 4: was only trying to help his fellow faggots with their harassment of foreigners.


Thanks for your
 time


In [25]:
generated_text_samples = base_model.generate(
    text_ids,
    max_length= 50,  
    do_sample=True,  
    top_k=100,
    top_p=0.92,
    temperature=0.8,
    repetition_penalty= 1.5,
    num_return_sequences= 5
)

for i, beam in enumerate(generated_text_samples):
  pretty_print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
  print()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0: was only trying to help the children, he said. "The mother was quite upset at first."
(Read more:
 http://www....)

 1: was only trying to help their local people. She was very interested in the area, she always liked
 what they were doing there and wanted more money."
 (Karen Brueckner/CBC) "We had a lot of success
 with women

 2: was only trying to help, but he was hurting. It wasn't all that long ago I had some kind of heart
 problem and it became very bad," she said.
"He asked me if we would like a second opinion on what is
 going

 3: was only trying to help others who couldn't afford it. But that wasn
"a serious problem," he
 said. "We've lost four in a row since 2009, but this is the first time we have not been able fully
 recover from one

 4: was only trying to help others who needed it.
This is a very specific case, but the most common
 problem we encounter in many cases of mental health issues among LGBT people are problems with their
 own bodies and relationships or la

In [26]:
def generate_n_text_samples(model, tokenizer, input_text, device, n_samples = 5):
    text_ids = tokenizer.encode(input_text, return_tensors = 'pt')
    text_ids = text_ids.to(device)
    model = model.to(device)

    generated_text_samples = model.generate(
        text_ids, 
        max_length= 100,  
        num_return_sequences= n_samples,
        no_repeat_ngram_size= 2,
        repetition_penalty= 1.5,
        top_p= 0.92,
        temperature= .85,
        do_sample= True,
        top_k= 125,
        early_stopping= True
    )
    gen_text = []
    for t in generated_text_samples:
        text = tokenizer.decode(t, skip_special_tokens=True)
        gen_text.append(text)

        return gen_text


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## ***Как для заголовков***

In [27]:
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}

# the new token is added to the tokenizer
num_added_toks = base_tokenizer.add_special_tokens(special_tokens_dict)

# the model config to which we add the special tokens
config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=base_tokenizer.bos_token_id,
                                    eos_token_id=base_tokenizer.eos_token_id,
                                    pad_token_id=base_tokenizer.pad_token_id,
                                    output_hidden_states=False)

# the pre-trained model is loaded with the custom configuration
base_model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

# the model embedding is resized
base_model.resize_token_embeddings(len(base_tokenizer))


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Embedding(50259, 768)

In [28]:
covid_sample['full_clear'] = bos + ' ' + covid_sample['full_clear'] + ' ' + eos

df_train, df_val = train_test_split(covid_sample, train_size = 0.9, random_state = 100)
print(f'There are {len(df_train)} headlines for training and {len(df_val)} for validation')


There are 9000 headlines for training and 1000 for validation


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
train_dataset = Dataset.from_pandas(df_train[['full_clear']])
val_dataset = Dataset.from_pandas(df_val[['full_clear']])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
def tokenize_function(examples):
    return base_tokenizer(examples['full_clear'], padding=True)


tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['full_clear'],
)
tokenized_val_dataset = val_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['full_clear'],
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Map (num_proc=5):   0%|          | 0/9000 [00:00<?, ? examples/s]

Map (num_proc=5):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [31]:
base_tokenizer.decode(tokenized_train_dataset['input_ids'][0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'<|endoftext|> you decided not to wear a mask because you listened to the insane ramblings of a lunatic mind at t <|EOS|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|>'

In [33]:
model_headlines_path = './model_headlines_news'

training_args = TrainingArguments(
    output_dir=model_headlines_path,          # output directory
    num_train_epochs=6,              # total # of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=model_headlines_path,            # directory for storing logs
    prediction_loss_only=True,
    save_steps=10000 
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=base_tokenizer,
        mlm=False
    )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
trainer = Trainer(
    model=base_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,         # training dataset
    eval_dataset=tokenized_val_dataset            # evaluation dataset
)
trainer.train()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,10.172600
1000,4.652400
1500,4.361800


TrainOutput(global_step=1692, training_loss=6.15411694393654, metrics={'train_runtime': 960.4484, 'train_samples_per_second': 56.224, 'train_steps_per_second': 1.762, 'total_flos': 1708523274240000.0, 'train_loss': 6.15411694393654, 'epoch': 6.0})

In [36]:
trainer.save_model()
base_tokenizer.save_pretrained(model_headlines_path)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


('./model_headlines_news/tokenizer_config.json',
 './model_headlines_news/special_tokens_map.json',
 './model_headlines_news/vocab.json',
 './model_headlines_news/merges.txt',
 './model_headlines_news/added_tokens.json')

In [37]:
model_headlines_path = './model_headlines_news'
headlines_tokenizer = GPT2Tokenizer.from_pretrained(model_headlines_path)
headlines_model = GPT2LMHeadModel.from_pretrained(model_headlines_path)
device = "cuda:0"
input_text = headlines_tokenizer.bos_token

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
text = "Covid19"
text1 = "Coronavirus"
text_ids = headlines_tokenizer.encode(text, return_tensors="pt")
text_ids1 = headlines_tokenizer.encode(text1, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
generated_text_samples = headlines_model.generate(text_ids, max_length=250, do_sample=True, top_k=100, top_p=0.92, temperature=0.85, repetition_penalty=1.75, num_return_sequences=20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [40]:
generated_text_samples1 = headlines_model.generate(text_ids1, max_length=250, do_sample=True, top_k=100, top_p=0.92, temperature=0.85, repetition_penalty=1.75, num_return_sequences=20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
for i in range(len(generated_text_samples)):
  print(f"{i+1}: {headlines_tokenizer.decode(generated_text_samples[i], skip_special_tokens=True)}")

1: Covid19 incidence in us as of 01 august 2020 487 confirmed cases 745 recovered 486 recoveries 1188 recovered deaths
2: Covid19 situation in germany continues to deterior
3: Covid19 testing centre in honduras tests positive for covid 19
4: Covid19 rate of covc in maharashtra reports highest spike of 662467 cases surpassing previous record for first time
5: Covid19 vaccine has proven to be a huge boost for many families in the developing world who do have their children
6: Covid19 is now the 2nd leading cause of death worldwide in a single day
7: Covid19 update for the mid 2020 saturday august 7th 68949 cases reported in total 1248 gmt
8: Covid19 vaccine trial in india
9: Covid19 vaccine is not a panacea for the problem of covids in particular as well it does contain deadly i
10: Covid19 vaccine in children
11: Covid19 is just the beginning of what will be a daunting challenge when it comes to handling coronavirus cases in all parts o
12: Covid19 is an exciting new finding which expla

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
for i in range(len(generated_text_samples1)):
  print(f"{i+1}: {headlines_tokenizer.decode(generated_text_samples1[i], skip_special_tokens=True)}")

1: Coronavirus is now the second most deadly infectious disease worldwide after
2: Coronavirus via
3: Coronavirus can damage the liver and kidney because of a lack o
4: Coronavirus covid19 cases in india total number of confirmed positive cases as on august 27 2020 24
5: Coronavirus covid19 cases in india rise to 489747
6: Coronavirus s work to fight infections in 2020
7: Coronavirus covid19 updates in april
8: Coronavirus situation in australia after coronation and pandemic
9: Coronavirus virus has been linked to a range
10: Coronavirus is a global pandemic problem that impacts over 500 million people worldwide according to the latest figures
11: Coronavirus covid19 death toll is still at a ppe level so it s possible that we may have crossed the threshold
12: Coronavirus i m the only one with a vaccine i know how to tell if you have it or not i love these i m
13: Coronavirus is the virus that causes severe disabilities such as mobility and
14: Coronavirus in the us has reached a new l

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
